In [1]:
from xgboost import XGBClassifier
import pandas as pd 
import wandb
from gpt3forchem.polymers.featurizer  import featurize_many
from gpt3forchem.polymers.data import get_data
from gpt3forchem.polymers.constants import TARGETS, FEATURES, CAT_TARGETS
from gpt3forchem.mofs.baseline.classification import XGBClassificationBaseline
from pycm import *
from sklearn.model_selection import train_test_split

import joblib

/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = get_data()

In [3]:
CAT_TARGETS

['deltaGmin_cat', 'A2_normalized_cat']

In [4]:
X = df[FEATURES]
y = df[CAT_TARGETS[0]]

In [5]:
model = XGBClassificationBaseline(524524)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9)

In [7]:
run = wandb.init(
    project="gpt-3",
    job_type="delta-g-predicttor",
    config={
        "train_size": len(X_train),
        "test_size": len(X_test),
        "features": FEATURES,
        "targets": [CAT_TARGETS[0]],
        "ds": "mof",
    },
)


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin


In [8]:
model.tune(X_train, y_train)

[I 2022-07-01 10:14:05,681] A new study created in memory with name: no-name-f21f7a82-ae5e-4585-a5a4-214bca3d0962
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/src/gpt3forchem/mofs/baseline/classification.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.05),
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/src/gpt3forchem/mofs/baseline/classification.py:40: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 1),
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/src/gpt3forchem/mofs/baseline/classi

In [ ]:
model.fit(X_train.values, y_train)

In [ ]:
test_predictions = model.predict(X_test)

In [ ]:
cm = ConfusionMatrix(y_test.values, test_predictions)

In [ ]:
print(cm)

Predict          large            medium           small            very large       very small       
Actual
large            29               12               1                9                0                

medium           10               41               11               0                0                

small            0                17               43               0                8                

very large       10               0                0                51               0                

very small       0                1                11               0                59               





Overall Statistics : 

95% CI                                                            (0.66232,0.7626)
ACC Macro                                                         0.88498
ARI                                                               0.46373
AUNP                                                              0.82075
AUNU                                  

In [ ]:
j